<a href="https://colab.research.google.com/github/elizabethyoo/team_9/blob/data%2FBFS_mf/Complie_sbp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import glob
import re
import numpy as no
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
files = glob.glob(f"/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/*.xlsx") 

In [4]:
emp_files = [x for x in files if 'empclass' in x]
st_sector_files = [x for x in files if '/national_state_sector' in x]
st_files = [x for x in files if '/national_state_' in x and not '/national_state_sector' in x]
naics3_files = [x for x in files if 'naics3' in x]
indexes_files = [x for x in files if 'indexes' in x]
msa_files = [x for x in files if 'top_50_msa' in x]
response_urr_files = [x for x in files if 'response_urr' in x]

In [5]:
st_sector_files[0]

'/content/drive/MyDrive/Group 9 Project Documents/Datasets/business_pulse/national_state_sector_26Apr20_02May20.xlsx'

In [29]:
def p2f(x):
    return float(x.strip('%'))/100

In [18]:
regex = r'\d+[A-Z][a-z]+\d+_\d+[A-Z][a-z]+\d+'  
period_str = re.search(regex, st_sector_files[0].split('/')[-1])[0]
print(period_str)

26Apr20_02May20


In [21]:
date_start_end = period_str.split('_')

In [23]:
date_start = pd.to_datetime(date_start_end[0], format= '%d%b%y')
date_end = pd.to_datetime(date_start_end[1], format= '%d%b%y')

In [30]:
df = pd.read_excel(st_sector_files[0],  converters={'ESTIMATE_PERCENTAGE':p2f, 'SE':p2f}).assign(DATE_START=date_start).assign(DATE_END= date_end)

In [31]:
df.head()

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE_START,DATE_END
0,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,0.514,0.0021,2020-04-26,2020-05-02
1,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,0.385,0.0027,2020-04-26,2020-05-02
2,-,-,1,"Overall, how has this business been affected b...",3,Little or no effect,0.076,0.0029,2020-04-26,2020-05-02
3,-,-,1,"Overall, how has this business been affected b...",4,Moderate positive effect,0.017,0.0008,2020-04-26,2020-05-02
4,-,-,1,"Overall, how has this business been affected b...",5,Large positive effect,0.008,0.0006,2020-04-26,2020-05-02


In [37]:
df.loc[((df['INSTRUMENT_ID']==1) & (df['NAICS_SECTOR']=='-')),:]

,ST,NAICS_SECTOR,INSTRUMENT_ID,QUESTION,ANSWER_ID,ANSWER_TEXT,ESTIMATE_PERCENTAGE,SE,DATE_START,DATE_END
0,-,-,1,"Overall, how has this business been affected b...",1,Large negative effect,0.514,0.0021,2020-04-26,2020-05-02
1,-,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,0.385,0.0027,2020-04-26,2020-05-02
2,-,-,1,"Overall, how has this business been affected b...",3,Little or no effect,0.076,0.0029,2020-04-26,2020-05-02
3,-,-,1,"Overall, how has this business been affected b...",4,Moderate positive effect,0.017,0.0008,2020-04-26,2020-05-02
4,-,-,1,"Overall, how has this business been affected b...",5,Large positive effect,0.008,0.0006,2020-04-26,2020-05-02
...,...,...,...,...,...,...,...,...,...,...
13027,WV,-,1,"Overall, how has this business been affected b...",1,Large negative effect,0.439,0.0716,2020-04-26,2020-05-02
13028,WV,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,0.417,0.0504,2020-04-26,2020-05-02
13062,WY,-,1,"Overall, how has this business been affected b...",1,Large negative effect,0.490,0.0652,2020-04-26,2020-05-02
13063,WY,-,1,"Overall, how has this business been affected b...",2,Moderate negative effect,0.343,0.0697,2020-04-26,2020-05-02


In [11]:
state_name_mapping = {
  "AL": "Alabama",
  "AK": "Alaska",
  "AZ": "Arizona",
  "AR": "Arkansas",
  "CA": "California",
  "CO": "Colorado",
  "CT": "Connecticut",
  "DE": "Delaware",
  "FL": "Florida",
  "GA": "Georgia",
  "HI": "Hawaii",
  "ID": "Idaho",
  "IL": "Illinois",
  "IN": "Indiana",
  "IA": "Iowa",
  "KS": "Kansas",
  "KY": "Kentucky",
  "LA": "Louisiana",
  "ME": "Maine",
  "MD": "Maryland",
  "MA": "Massachusetts",
  "MI": "Michigan",
  "MN": "Minnesota",
  "MS": "Mississippi",
  "MO": "Missouri",
  "MT": "Montana",
  "NE": "Nebraska",
  "NV": "Nevada",
  "NH": "New Hampshire",
  "NJ": "New Jersey",
  "NM": "New Mexico",
  "NY": "New York",
  "NC": "North Carolina",
  "ND": "North Dakota",
  "OH": "Ohio",
  "OK": "Oklahoma",
  "OR": "Oregon",
  "PA": "Pennsylvania",
  "RI": "Rhode Island",
  "SC": "South Carolina",
  "SD": "South Dakota",
  "TN": "Tennessee",
  "TX": "Texas",
  "UT": "Utah",
  "VT": "Vermont",
  "VA": "Virginia",
  "WA": "Washington",
  "WV": "West Virginia",
  "WI": "Wisconsin",
  "WY": "Wyoming"
}

# TO DO

1. The state name is in abbreviation. Data structure/ value for state name is different from the household data. Therefore, a conversion of the state name is needed. (Either on the small business dataset or the household dataset)

2. Select features that can be used as an indicator for business recovery.

3. from the household datasets, find interesting variables that might be associated with the business recovery. 